In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-reviews/texts.csv


In [17]:
import re
import random
from typing import List, Tuple

def create_synthetic_space_data(
    texts: List[str],
    max_samples: int = 10000
) -> List[Tuple[str, List[int]]]:
    """
    Генерирует синтетические примеры для задачи восстановления пробелов.

    Args:
        texts: список исходных предложений на русском языке,
               а также с возможными английскими словами, числами, запятыми и апострофами.
        max_samples: максимальное количество генерируемых примеров.

    Returns:
        Список кортежей (text_no_spaces, space_positions):
            text_no_spaces — строка без пробелов;
            space_positions — список индексов, перед которыми должен быть пробел.
    """
    synthetic_data = []
    # Регулярное выражение: сохраняем кириллицу, латиницу, цифры, пробелы, запятые и апострофы
    pattern = re.compile(r"[^а-яёА-ЯЁa-zA-Z0-9\s,\'’]")
    for text in texts:
        # Очищаем текст, оставляя нужные символы, переводим в нижний регистр
        clean = pattern.sub("", text).strip()[:-8]
        words = clean.split()
        # Фильтрация по длине предложения
        if len(words) < 3 or len(words) > 20:
            continue

        # Склеиваем слова без пробелов
        text_no_spaces = "".join(words)

        # Вычисляем позиции пробелов
        positions = []
        pos = 0
        for i, w in enumerate(words):
            if i > 0:
                positions.append(pos)
            pos += len(w)

        synthetic_data.append((text_no_spaces, positions))
        if len(synthetic_data) >= max_samples:
            break

    return synthetic_data

In [26]:
with open('/kaggle/input/data-reviews/texts.csv', 'r') as f:
    f = f.readlines()
print(len(f))
data = create_synthetic_space_data(f[1:], max_samples=9999999)

101893


In [27]:
data[1]

('товарнепришел,продавецпродлилзащитубезмоегосогласияотпродавцаодниобещания',
 [5, 7, 14, 22, 29, 35, 38, 43, 51, 53, 61, 65])

In [28]:
len(data)

63666

In [83]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from typing import List, Tuple

class SpaceRestorationDataset(Dataset):
    """
    Класс-датасет для задачи восстановления пробелов:
    - examples: список кортежей (текст_без_пробелов, позиции_пробелов)
    - для каждого примера создаём метки на уровне сабтокенов с учётом символных позиций
    """
    def __init__(
        self,
        examples: List[Tuple[str, List[int]]],
        tokenizer_name: str = "ai-forever/ruRoberta-large",
        max_length: int = 128
    ):
        # Сохраняем примеры
        self.examples = examples
        # Инициализируем fast-токенизатор Roberta с add_prefix_space=True,
        # чтобы корректно работать с предтокенизированными строками
        self.tokenizer = AutoTokenizer.from_pretrained(
            tokenizer_name, use_fast=True, add_prefix_space=True
        )
        # Максимальная длина последовательности для паддинга/транкации
        self.max_length = max_length

    def __len__(self):
        # Количество примеров
        return len(self.examples)
    
    def __getitem__(self, idx: int):
        # Получаем текст без пробелов и список позиций пробелов
        text_ns, space_positions = self.examples[idx]

        # 1. Создаём array нулей длиной текста, затем отмечаем char-level метки
        char_labels = [0] * len(text_ns)
        for p in space_positions:
            if 0 <= p < len(char_labels):
                char_labels[p] = 1

        # 2. Токенизируем строку целиком, запрашивая offsets для выравнивания меток
        enc = self.tokenizer(
            text_ns,
            return_offsets_mapping=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_length
        )
        input_ids = enc["input_ids"]             # индексы сабтокенов
        attention_mask = enc["attention_mask"]   # маска паддинга
        offsets = enc["offset_mapping"]          # соответствие токен→спану символов

        # 3. Выравниваем метки на уровне токенов:
        #    если в диапазоне одного токена есть хотя бы один char_label=1 → метка=1
        token_labels = []
        for (start, end) in offsets:
            if start == end:
                # [CLS], [SEP], паддинг
                token_labels.append(0)
            else:
                # проверяем все символы в диапазоне
                label = 0
                for i in range(start, end):
                    if char_labels[i] == 1:
                        label = 1
                        break
                token_labels.append(label)

        # 4. Конвертируем всё в тензоры и возвращаем в виде словаря
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(token_labels, dtype=torch.long),
        }


from sklearn.model_selection import train_test_split

# data — это список примеров [(текст_без_пробелов, позиции_пробелов), ...]
train_ex, val_ex = train_test_split(data, test_size=0.2, random_state=42)

# Инициализируем датасеты с тем же токенизатором для train и val
train_dataset = SpaceRestorationDataset(
    train_ex,
    tokenizer_name="ai-forever/ruRoberta-large",
    max_length=128
)
val_dataset = SpaceRestorationDataset(
    val_ex,
    tokenizer_name="ai-forever/ruRoberta-large",
    max_length=128
)

# Создаём DataLoader:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Проверка форматов первого батча
batch = next(iter(train_loader))
print("input_ids:", batch["input_ids"].shape)        # (16, 128)
print("attention_mask:", batch["attention_mask"].shape)  # (16, 128)
print("labels:", batch["labels"].shape)              # (16, 128)


torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])


In [43]:
!pip install pytorch-crf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [87]:
torch.cuda.empty_cache()

In [88]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from transformers import AutoModel
from torchcrf import CRF
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support
from tqdm.auto import tqdm

class RuBERTCRFSpaceRestoration(nn.Module):
    """
    Модель RuRoBERTa + CRF для восстановления пропущенных пробелов.
    Архитектура:
      - Pretrained Transformer (ruRoberta-large) для получения контекстных эмбеддингов.
      - Dropout для регуляризации.
      - Linear-классификатор на hidden_size → 2 (0/1 метки).
      - CRF-слой для моделирования зависимостей между позициями пробелов.
    """
    def __init__(
        self,
        model_name: str = "ai-forever/ruRoberta-large",
        num_labels: int = 2
    ):
        super().__init__()
        # 1. Pretrained Transformer
        self.bert = AutoModel.from_pretrained(model_name)
        # 2. Dropout перед классификатором
        self.dropout = nn.Dropout(0.1)
        # 3. Классификатор для предсказания эмиссий
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        # 4. CRF-слой для последовательной разметки
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        # Вычисляем эмбеддинги
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs.last_hidden_state)
        # Получаем эмиссии (логиты) для CRF
        emissions = self.classifier(sequence_output)

        if labels is not None:
            # Обратное распространение: CRF loss (отрицательный log-likelihood)
            loss = -self.crf(
                emissions,
                labels,
                mask=attention_mask.bool(),
                reduction="mean"
            )
            return loss
        else:
            # Инференс: decode лучших последовательностей меток
            return self.crf.decode(
                emissions,
                mask=attention_mask.bool()
            )

def calculate_f1(preds, labels, mask):
    """
    Вычисление precision, recall и F1 по позициям пробелов.
    Собираем все метки и предсказания по не-паддинговым токенам.
    """
    pred_flat, true_flat = [], []
    for pred_seq, true_seq, m in zip(preds, labels, mask):
        for p, t, mm in zip(pred_seq, true_seq.tolist(), m.tolist()):
            if mm == 1:  # учитываем только реальные токены
                pred_flat.append(p)
                true_flat.append(t)
    p, r, f1, _ = precision_recall_fscore_support(
        true_flat, pred_flat,
        average="binary",
        pos_label=1
    )
    return p, r, f1

def train_epoch(model, dataloader, optimizer, device):
    """
    Одна эпоха обучения:
      - model.train() для включения dropout.
      - Проход в tqdm для визуального контроля прогресса.
      - Накопление и вывод среднего loss.
    """
    model.train()
    total_loss = 0.0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)
    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Прямой проход и вычисление loss
        loss = model(input_ids, attention_mask, labels)
        optimizer.zero_grad()
        loss.backward()       
        optimizer.step()      

        total_loss += loss.item()
        avg_loss = total_loss / (progress_bar.n + 1)
        progress_bar.set_postfix(loss=f"{avg_loss:.4f}")

    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    """
    Оценка на валидации:
      - model.eval() для отключения dropout.
      - Сбор предсказаний без градиентов.
      - Возврат precision, recall и F1.
    """
    model.eval()
    all_preds, all_labels, all_mask = [], [], []
    progress_bar = tqdm(dataloader, desc="Validation", leave=False)
    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            preds = model(input_ids, attention_mask)
            all_preds.extend(preds)
            all_labels.extend(labels.cpu())
            all_mask.extend(attention_mask.cpu())
            progress_bar.update(0)

    return calculate_f1(all_preds, all_labels, all_mask)


    
from sklearn.model_selection import train_test_split

# Гиперпараметры
MODEL_NAME = "ai-forever/ruRoberta-large"
BATCH_SIZE = 16
LR = 2e-5
EPOCHS = 3
MAX_LEN = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Предполагается, что data — это список (text_no_space, positions)
train_ex, val_ex = train_test_split(data[:20000], test_size=0.2, random_state=42)
# почему data[:20000] - обучение на полном датасете занимало слишком много времени (45 мин на эпоху)

In [89]:

# Даталоадеры
train_ds = SpaceRestorationDataset(train_ex, tokenizer_name=MODEL_NAME, max_length=MAX_LEN)
val_ds = SpaceRestorationDataset(val_ex, tokenizer_name=MODEL_NAME, max_length=MAX_LEN)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)


In [90]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RuBERTCRFSpaceRestoration(MODEL_NAME).to(device)
optimizer = AdamW(model.parameters(), lr=LR)



Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [91]:
device

device(type='cuda')

In [92]:
best_f1 = 0.0
for epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    p, r, f1 = evaluate(model, val_loader, device)
    print(f"Epoch {epoch+1}/{EPOCHS} — Train Loss: {train_loss:.4f} | Val F1: {f1:.4f} (P={p:.4f},R={r:.4f})")
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), "best_space_restoration.pth")
        print(f"Saved best model with F1={best_f1:.4f}")


Training:   0%|          | 0/1000 [00:00<?, ?it/s]

Validation:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1/3 — Train Loss: 2.8866 | Val F1: 0.9602 (P=0.9475,R=0.9732)
Saved best model with F1=0.9602


Training:   0%|          | 0/1000 [00:00<?, ?it/s]

Validation:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 2/3 — Train Loss: 2.0639 | Val F1: 0.9612 (P=0.9420,R=0.9812)
Saved best model with F1=0.9612


Training:   0%|          | 0/1000 [00:00<?, ?it/s]

Validation:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 3/3 — Train Loss: 2.5323 | Val F1: 0.0000 (P=0.0000,R=0.0000)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [93]:
evaluate(model, val_loader, device)

Validation:   0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0, 0.0, 0.0)

In [126]:
# Проверка на количество меток, почему то не получилось обучить модель 
def check_label_distribution(dataloader):
    label_counts = {0: 0, 1: 0}
    for batch in dataloader:
        labels = batch['labels']
        mask = batch['attention_mask']
        for label_seq, mask_seq in zip(labels, mask):
            for l, m in zip(label_seq, mask_seq):
                if m == 1:  # только не-паддинг токены
                    label_counts[l.item()] += 1
    print("Label distribution:", label_counts)
    return label_counts

check_label_distribution(train_loader)
check_label_distribution(val_loader)

Label distribution: {0: 231448, 1: 140108}
Label distribution: {0: 57304, 1: 34983}


{0: 57304, 1: 34983}

In [120]:
data = {}
with open('/kaggle/input/avito-requests/dataset_1937770_3.txt', 'r') as f:
    f = f.readlines()
    
    columns = f[0].strip().split(',')
    for i in columns:
        data[i] = []

    for line in f[1:]:
        # print(line)
        res = line.strip().split(',', 1)
        data[columns[0]].append(res[0])
        data[columns[1]].append(res[1])


In [121]:
import pandas as pd
df = pd.DataFrame(data)
df['predicted_positions'] = df['id'].map(lambda x: [])

df.head()

,id,text_no_spaces,predicted_positions
0,0,куплюайфон14про,[]
1,1,ищудомвПодмосковье,[]
2,2,сдаюквартирусмебельюитехникой,[]
3,3,новыйдивандоставканедорого,[]
4,4,отдамдаромкошку,[]


In [122]:
# 1. Загрузка модели и токенизатора
MODEL_NAME = "ai-forever/ruRoberta-large"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, use_fast=True, add_prefix_space=True
)
model = RuBERTCRFSpaceRestoration(MODEL_NAME)
model.load_state_dict(torch.load("best_space_restoration.pth", map_location=DEVICE))
model.to(DEVICE)
model.eval()

# 2. Функция для предсказания позиций пробелов
def predict_positions(text_ns: str) -> list[int]:
    # Токенизация
    enc = tokenizer(
        text_ns,
        return_offsets_mapping=True,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    input_ids = enc["input_ids"].to(DEVICE)
    attention_mask = enc["attention_mask"].to(DEVICE)
    offsets = enc["offset_mapping"][0].tolist()

    # Предсказание
    with torch.no_grad():
        preds = model(input_ids, attention_mask)[0]  # берем первый (и единственный) пример

    # Собираем позиции по символам
    positions = []
    for token_idx, label in enumerate(preds):
        if label == 1:
            start, end = offsets[token_idx]
            # добавляем позицию старта токена
            positions.append(start)

    # Фильтруем и сортируем
    positions = sorted(set(p for p in positions if p < len(text_ns)))
    return positions


# Заполняем predicted_positions
df["predicted_positions"] = df["text_no_spaces"].apply(predict_positions)

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [124]:
df

,id,text_no_spaces,predicted_positions
0,0,куплюайфон14про,"[5, 10, 12]"
1,1,ищудомвПодмосковье,"[0, 2, 6, 7]"
2,2,сдаюквартирусмебельюитехникой,"[3, 12, 20]"
3,3,новыйдивандоставканедорого,"[5, 8, 16, 19]"
4,4,отдамдаромкошку,"[5, 10]"
...,...,...,...
1000,1000,Янеусну.,"[1, 3]"
1001,1001,Весна-яуженегреюпио.,"[5, 6, 7, 9, 11, 16]"
1002,1002,Весна-скоровырастеттрава.,"[5, 6, 13, 17]"
1003,1003,"Весна-выпосмотрите,каккрасиво.","[5, 6, 8, 19, 22]"


In [127]:
# Оставляем только нужные столбцы
output_df = df[["id", "predicted_positions"]]

# скачать файл csv 
output_df.to_csv("predicted_positions.csv", index=False, encoding="utf-8", header=["id", "predicted_positions"])
